# Item 21. 일반적인 프로퍼티 패턴은 프로퍼티 위임으로 만들어라

## 프로퍼티 위임?

Item 16에서 프로퍼티는 클래스의 상태 정보를 조작하거나 조회하는 일종의 래퍼 로서 정의했다.

이러한 프로퍼티 조회 혹은설정은 다양한 패턴이 존재하고 이 반복되는 패턴을 추출하여 재사용 가능하게 하는 것이 코틀린의 프로퍼티 위임이다.

## Standard Property Delegation

kotlin stdlib에서 제공해주는 기본 프로퍼티 델리게이터

- lazy
- Delegates.observable
- Delegates.vetoable
- Delegates.notNull

### lazy?

코틀린은 크게 두가지 지연 초기화 기법을 제공한다.

- Late init : 필요할 때 초기화하고 사용할 수 있음. 초기화하지 않고 쓰면 throw ex

In [ ]:
class Rectangle {
    lateinit var area: Area
    fun initArea(param: Area): Unit {
        this.area = param
    }
}

class Area(val value: Int)

val rectangle = Rectangle()
rectangle.initArea(Area(10)) // 초기화안하면 예외터짐!
println(rectangle.area.value)

- lateinit을 사용

자바의 클래스 멤버 변수 지연 초기화할때랑 동일한 방식으로 스프링 사용시 @Autowired 등 필드 주입할 필요가 있을때 주로 사용

프리미티브 타입은 사용 불가, var 에만 사용가능하며 커스텀 getter, setter 불가능
NonNull 프로퍼티만 사용가능

- Lazy init : 변수를 선언할 때 초기화 코드도 함께 정의

In [ ]:
val balance : Int by lazy {
    println("Setting balance!")
    100
}

val 도 가능, 프리미티브 타입 가능, Non-null, nullable 둘다 가능
**쓰레드 세이프함!**


### NotNull?

deletgation을 통해 NotNull을 쓸경우도 프로퍼티 지연 로딩이 가능하고, 호출시 Null 체킹을 하므로 lateInit과 비슷한 의도로 사용가능하다

하지만 위임방식이므로 val, 프리미티브 타입에 가능하고 쓰레드 세이프한 장점이있다.

반면 lateInit에 비해 추가적인 오버헤드를 감안해야한다.


In [14]:
import kotlin.properties.Delegates

var nonNullString: String by Delegates.notNull<String>()
nonNullString = "text"
println("Non null value is: ${nonNullString}")

Non null value is: text


### observable

setter 호출시마다 observable로 작성한 콜백을 실행하는 방식

### vetoable

setter 호출시 vetoable로 각성한 콜백을 실행하여 t/f 를 반환하고 반환값이 f면 에러를 발생시키는 일종의 validation 기법



## 실무적으로?

프로퍼티 위임을 처음에 학습하면 주로 엔티티에 쓰고 싶은 욕심이 생기는데 안타깝게도 jpa 엔티티는 프로퍼티 위임을 사용할 수 없다.

클린아키텍처를 적극적으로 수용하여 도메인레이어와 영속레이어를 분리시키고 정말 순수한 도메인을 유지한다면 도메인 레이어의 엔티티에 `vetoable` 같은 위임을 사용하여

invariant 한 검증등은 할수 있지만 그렇지 않은 프로젝트에서 적극 사용하기엔 부담스럽다.

현실적으로는 dto 들에 일부 적용하는정도...
